# Evaluate Multi Agent System Simulation

This notebook evaluates the accuracy and performance of the three-agent system of banks made in this repository with real world data.

In [ ]:
import os

import pandas as pd
import requests
from dotenv import load_dotenv

from agentomics.common.data_structures import ThreeBankGlobalState
from agentomics.common.types import NonnegPercent, Percent
from scripts.economic_simulations.three_banks import simulate

## Evaluation with initial economic variables only

This section tests the multi-agent system pipeline with just the initialization of the economic variables. In this case, the agents have no prior information on their behavior in previous quarters. This evaluation thus relies solely on the LLM's training data to make good decisions and properly propagate changes in economic variables.

Hypothesis: the system will not match actual proceedings as the bank agents make the best decisions on all the information it knows (which is vastly more in some cases than some of the case studies' banks would have had access to) and the economic agent simply works off of these banks being representative of other similar banks.

In [ ]:
# Import GDP Growth Rate from FRED
load_dotenv()
# FRED API details
fred_api_url = "https://api.stlouisfed.org/fred/series/observations"
fred_api_key = os.getenv("FRED_API_KEY")
gdp_params = {
    "series_id": "A191RL1Q225SBEA",  # GDP growth rate series ID
    "api_key": fred_api_key,
    "file_type": "json",  # JSON format
    "frequency": "q",  # Quarterly frequency
    "realtime_start": "2015-01-01",  # Adjust as needed
    "realtime_end": "2024-01-01"  # Adjust as needed
}

# Fetch data from FRED API
gdp_response = requests.get(fred_api_url, params=gdp_params)
gdp_data = gdp_response.json()

gdp_observations = gdp_data["observations"]
df = pd.DataFrame(gdp_observations)

df["date"] = pd.to_datetime(df["date"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")
df = df[["date", "value"]].rename(columns={"value": "Quarterly GDP Growth Rate"})

df["Quarterly GDP Growth Rate"] /= 100.0

In [ ]:
# Import Quarterly Unemployment Rate and Inflation Rate (CPI) from FRED

# Unemployment Rate parameters
unemployment_params = {
    "series_id": "UNRATE",  # Unemployment rate series ID
    "api_key": fred_api_key,
    "file_type": "json",  # JSON format
    "frequency": "q",  # Quarterly frequency
    "start_date": "2010-01-01",  # Adjust as needed
    "end_date": "2024-01-01"  # Adjust as needed
}

# CPI Inflation Rate parameters
inflation_params = {
    "series_id": "CPIAUCSL",  # CPI series ID
    "api_key": fred_api_key,
    "file_type": "json",  # JSON format
    "frequency": "q",  # Quarterly frequency
    "start_date": "2015-01-01",  # Adjust as needed
    "end_date": "2024-01-01"  # Adjust as needed
}

# Fetch Unemployment Rate data from FRED API
unemployment_response = requests.get(fred_api_url, params=unemployment_params)
unemployment_data = unemployment_response.json()

unemployment_observations = unemployment_data["observations"]
unemployment_df = pd.DataFrame(unemployment_observations)

unemployment_df["date"] = pd.to_datetime(unemployment_df["date"])
unemployment_df["value"] = pd.to_numeric(unemployment_df["value"], errors="coerce")
unemployment_df["value"] /= 100.0
unemployment_df = unemployment_df[["date", "value"]].rename(columns={"value": "Quarterly Unemployment Rate"})

# Fetch Inflation Rate (CPI) data from FRED API
inflation_response = requests.get(fred_api_url, params=inflation_params)
inflation_data = inflation_response.json()

inflation_observations = inflation_data["observations"]
inflation_df = pd.DataFrame(inflation_observations)

inflation_df["date"] = pd.to_datetime(inflation_df["date"])
inflation_df["value"] = pd.to_numeric(inflation_df["value"], errors="coerce")
inflation_df = inflation_df[["date", "value"]].rename(columns={"value": "CPI"})

# Compute quarterly inflation rate as percent change
inflation_df["Quarterly Inflation Rate"] = inflation_df["CPI"].pct_change()

# Merging all data into a single DataFrame
merged_df = df.merge(unemployment_df, on="date", how="outer").merge(
    inflation_df[["date", "Quarterly Inflation Rate"]], on="date", how="outer"
)

In [ ]:
# Get the largest non-NaN contiguous block

# Identify rows without any NaNs
non_nan_mask = merged_df.notnull().all(axis=1)

# Create a group identifier for contiguous True blocks
merged_df["block"] = (non_nan_mask != non_nan_mask.shift()).cumsum()

# Filter only valid blocks where rows contain no NaNs
valid_blocks = merged_df.loc[non_nan_mask]

# Find the largest contiguous block
largest_block_id = valid_blocks["block"].value_counts().idxmax()
largest_block = merged_df[merged_df["block"] == largest_block_id].drop(columns=["block"])

# Display the largest contiguous sub-dataframe without NaNs
largest_block = largest_block.tail(10)
print(largest_block)

In [ ]:
# Set up data to put into simulation
N = len(largest_block)
K = 5  # We take the first 5 rows as context for the economic variables

sub_df = largest_block.head(N)
gdp_input = list(map(Percent, sub_df["Quarterly Inflation Rate"].tolist()))
unemployment_input = list(map(NonnegPercent, sub_df["Quarterly Unemployment Rate"].tolist()))
inflation_input = list(map(Percent, sub_df["Quarterly Inflation Rate"].tolist()))

globals = ThreeBankGlobalState()
globals.economic_variables.gdp_growth_rate.set_array(gdp_input)
globals.economic_variables.unemployment_rate.set_array(unemployment_input)
globals.economic_variables.inflation_rate.set_array(inflation_input)
globals.number_of_quarters_to_simulate = N - K

simulate(globals, "groq/llama3.1-70b-versatile")

In [ ]:
print(globals.print_subfields())
globals.to_csv("output/notebook_output.csv")